# Export-Third-Parties

## Token genaration and api management

In [1]:
#@title Import modules
import requests, base64
import re
import json as js
import pandas as pd
from io import StringIO
import numpy as np
import csv
import copy




In [2]:
#@title Token generation
#@markdown This code calls the token end point with client-id and secret

def login():
    cfg = pd.read_csv("/content/config.csv")
    client_id = cfg['client_id'].values[0]
    client_secret = cfg['client_secret'].values[0]
    token_endpoint = cfg['token_url'].values[0]
    
    data = {'grant_type': 'client_credentials'}
    
    userpass = client_id + ':' + client_secret
    encoded_u = base64.b64encode(userpass.encode()).decode()
    auth_header = "Basic " + encoded_u
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': auth_header
    }

    r = requests.post(token_endpoint, headers=headers, data=data)
    response = r.json()

    if (r.status_code == 200):
        token = response['access_token']
        # Put token in the session
        print(token)
    return token

In [3]:
#@title Get Results 
#@markdown This code calls the end point to retrieve data
def get_results(token, request):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.get(base_url + request, headers=headers)
        if 200 <= result.status_code < 300:
          return result.text
        else:
          print(result.text)
          return  
    except:
        return

In [4]:
#@title Post results  
#@markdown This code allows to POST data in payload or as a stream
def post_results(token, request, payload, files):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.post(base_url + request, headers=headers, data=payload, files = files)
        if 200 <= result.status_code < 300:
          json_data = js.loads(result.text)
          return json_data
        else:
          print(result.text)
          return
    except:
        return

In [5]:
#@title Launch report
#@markdown Code to manage report launch and retrieval of data in one function
#@markdown It launchs the task and waits until completion
def run_report(token, report):
    try:
        result = post_results(token, '/v1/process-templates/'+ report +'/run',"","")
        taskId = result[0]['taskId']
        while True:
            result = get_results(token, '/v1/process-templates/'+ taskId +'/status')
            if result == "Pending" or result == "In progress":
                print ('.', end='')
            else:
                print ('\n' + result)
            if result == "Warning" or result == "Complete" or result == "Error" or result == "Cancelled":
                break
                time.sleep(1)
        print('\n')
        result = get_results(token, '/v1/process-templates/'+ report + '/files?taskId=' + taskId )
        #print(result)
        return result
    except:
        return

## Tools for working with result


In [6]:
#@title Remove empty values in json

def remove_empty_elements(d):
    """recursively remove empty lists, empty dicts, or None elements from a dictionary"""

    def empty(x):
        return x is None or x == {} or x == []

    if not isinstance(d, (dict, list)):
        return d
    elif isinstance(d, list):
        return [v for v in (remove_empty_elements(v) for v in d) if not empty(v)]
    else:
        return {k: v for k, v in ((k, remove_empty_elements(v)) for k, v in d.items()) if not empty(v)}

In [7]:
#@title Convert csv to json

def convert_toJson(df):
  result = df.to_json(orient='records')
  parsed = js.loads(result, parse_int=int)
  return parsed

In [8]:
#@title Compare results

def compare_results(df1, df2):
  df_20 = pd.DataFrame()
  x = True
  for x in df1.index:
    if not ((df2['code'] == df1['Code'][x]) & (df2['name'] == df1['Name'][x])).any():
      print('Compared 20 rows, results are different')
      x = False
      break
    else: 
      df_20.append(df2, ignore_index = True)
  if x:
    print('Compared 20 rows, results match')
  display(df_20)
  return df_20

In [9]:
#@title Show results

def show_results(df, json, type, showEmptyValuesInJson):
  df2 = copy.deepcopy(df)
  if type == 'csv':
    df2.replace(np.nan, '-', inplace = True)
    display(df2)
  if type == 'json':
    parsed = convert_toJson(df) if json is None else json
    if not showEmptyValuesInJson:
      parsed = remove_empty_elements(parsed)
    print(js.dumps(parsed, indent=2, ensure_ascii=True) )

## Connection

In [ ]:
#@title <font color='lime'>Connect</font>
print('token:')
token = login()

## Exporting third-parties

In [ ]:
#@title <font color='lime'>Launch report on third-parties through Process Template</font>

#@markdown #### Choose content type from dropdown:
type = 'csv' #@param ['csv', 'json']
#@markdown #### Choose API to retrieve report:
thirdParties = True#@param{type: 'boolean'}
processTemplates = True#@param{type: 'boolean'}
#@markdown #### Options:
compareResults = True#@param{type: 'boolean'}
showEmptyValuesInJson = True#@param{type: 'boolean'}
doNotDisplayAccountInformation = True#@param{type: 'boolean'}

if processTemplates:
  res1 = run_report(token,'SU11')
  buffer = StringIO(res1)
  df1 = pd.read_csv (buffer,dtype=object, delimiter = ";", usecols=[1,2,3,5,6,8,10,17,30,32,36,37,38,61,69,70,72,73,78], names = ["Data type","Code","Name","Creditor","Debtor","Other identifier type","Non resident","Country code","Account.Country_code","Account.Name","Account.Currency","Account.ID_type","Account ID","Account.Used_by","Limit currency","Transaction entry limit","Account nature","Transaction max. number","Additional correspondent.One-time correspondent"],header=None, skiprows=[0])
  pd.set_option('display.max_colwidth', None)

if thirdParties:
  res2 = get_results(token, '/v1/third-parties')
  ppjson = js.loads(res2, parse_int=int)  
  df2 = pd.json_normalize(ppjson['results']).filter(items=['uuid', 'code', 'name', 'creationDate', 'updateDate', 'activeStatus', 'businessIdentifiers.otherIdentifierType', 'address.country.code'], axis =1)


df1_T = df1.loc[df1['Data type'].str.contains('T')].filter(regex = '^((?!(Account|Additional correspondent)).)*$', axis = 1)
df1_T.drop_duplicates(subset=['Code'], inplace = True)
if compareResults:
  if thirdParties and processTemplates:
    compare_results(df1_T.head(20), df2)
    print('\033[31m Results from process template: ')
    show_results(df1_T.head(20), None, type, showEmptyValuesInJson)
    print('\033[32m Results from third parties: ')
    df2_copy = copy.deepcopy(df2)
    show_results(df2_copy[df2_copy['code'].isin(df1_T.Code.head(20))], ppjson['results'], type, showEmptyValuesInJson)
  elif thirdParties:
    print('Results can be compared when both third-parties and process-templates are chosen.')
    print('\033[32m Results from third parties: ')
    show_results(df2, ppjson['results'], type, showEmptyValuesInJson)
  elif processTemplates:
    print('Results can be compared when both third-parties and process-templates are chosen.')
    print('\033[31m Results from process template: ')
    show_results(df1_T, None, type, showEmptyValuesInJson) if doNotDisplayAccountInformation else show_results(df1, None, type, showEmptyValuesInJson)
  else:
    print('Results can be compared when both third-parties and process-templates are chosen.')
else:
  if thirdParties:
    print('\033[32m Results from third parties: ')
    show_results(df2, ppjson['results'], type, showEmptyValuesInJson)
  if processTemplates:
    print('\033[31m Results from process template: ')
    show_results(df1_T, None, type, showEmptyValuesInJson) if doNotDisplayAccountInformation else show_results(df1, None, type, showEmptyValuesInJson)
  
  

